                                   Predicting Boston Housing Prices

## Import required packages.

In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

from dmba import regressionSummary, exhaustive_search, forward_selection, backward_elimination
from dmba import adjusted_r2_score, AIC_score, BIC_score

import matplotlib.pylab as plt
%matplotlib inline

## Upload data set for analysis. Explore, clean, and pre-process data. 

In [2]:
# Create data frame from the original data set.  
boston_df = pd.read_csv('BostonHousing.csv')

# Determine dimensions of dataframe. 
print('BostonHousing Dimensions:', boston_df.shape)


BostonHousing Dimensions: (506, 14)


In [3]:
# Display the column names.
print('Original Column Titles')
boston_df.columns

Original Column Titles


Index(['CRIME', 'ZONE', 'INDUST', 'CHAR RIV', 'NIT OXIDE', 'ROOMS', 'AGE',
       'DISTANCE', 'RADIAL', 'TAX', 'ST RATIO', 'LOW STAT', 'MVALUE',
       'C MVALUE'],
      dtype='object')

In [4]:
# Make column titles (variable names) as one word and 
# without blank. 
boston_df.columns = [s.strip().replace(' ', '_') for s in boston_df.columns]
print('Converted One-Word Titles')
boston_df.columns

Converted One-Word Titles


Index(['CRIME', 'ZONE', 'INDUST', 'CHAR_RIV', 'NIT_OXIDE', 'ROOMS', 'AGE',
       'DISTANCE', 'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'MVALUE',
       'C_MVALUE'],
      dtype='object')

In [5]:
# Display column data types in the data frame for regression analysis.
boston_df.dtypes

CRIME        float64
ZONE         float64
INDUST       float64
CHAR_RIV      object
NIT_OXIDE    float64
ROOMS        float64
AGE          float64
DISTANCE     float64
RADIAL         int64
TAX            int64
ST_RATIO     float64
LOW_STAT     float64
MVALUE       float64
C_MVALUE      object
dtype: object

In [6]:
# Convert object variables  into dummy variables.
# Use drop_first=True to drop the first dummy variable.
boston_df = pd.get_dummies(boston_df, prefix_sep='_', 
                            drop_first=True)
# Disply updated data types.  
boston_df.dtypes

CRIME           float64
ZONE            float64
INDUST          float64
NIT_OXIDE       float64
ROOMS           float64
AGE             float64
DISTANCE        float64
RADIAL            int64
TAX               int64
ST_RATIO        float64
LOW_STAT        float64
MVALUE          float64
CHAR_RIV_Y        uint8
C_MVALUE_Yes      uint8
dtype: object

In [7]:
np.round(boston_df.describe(), decimals=2)

,CRIME,ZONE,INDUST,NIT_OXIDE,ROOMS,AGE,DISTANCE,RADIAL,TAX,ST_RATIO,LOW_STAT,MVALUE,CHAR_RIV_Y,C_MVALUE_Yes
count,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00
mean,3.61,11.36,11.14,0.55,6.28,68.57,3.80,9.55,408.24,18.46,12.65,22.53,0.07,0.17
std,8.60,23.32,6.86,0.12,0.70,28.15,2.11,8.71,168.54,2.16,7.14,9.20,0.25,0.37
min,0.01,0.00,0.46,0.38,3.56,2.90,1.13,1.00,187.00,12.60,1.73,5.00,0.00,0.00
25%,0.08,0.00,5.19,0.45,5.89,45.02,2.10,4.00,279.00,17.40,6.95,17.02,0.00,0.00
50%,0.26,0.00,9.69,0.54,6.21,77.50,3.21,5.00,330.00,19.05,11.36,21.20,0.00,0.00
75%,3.68,12.50,18.10,0.62,6.62,94.07,5.19,24.00,666.00,20.20,16.96,25.00,0.00,0.00
max,88.98,100.00,27.74,0.87,8.78,100.00,12.13,24.00,711.00,22.00,37.97,50.00,1.00,1.00


## Develop multiple linear regression model and make predictions. 

In [8]:
# Develop outcome. 
outcome = 'MVALUE'

# Develop predictor variables. 
predictors = [s for s in boston_df.columns if s not in outcome]

# Identify X and y variables for regression and partition data
# using 70% of records for training and 30% for validation 
# (test_size=0.3). 
X = boston_df[predictors]
y = boston_df[outcome]
train_X, valid_X, train_y, valid_y = train_test_split(X, y, 
                            test_size=0.3, random_state=1)

# Create multiple linear regression model using X and y
# and LinearRegression() function from sklearn (skikit-learn) 
# library.
boston_lm = LinearRegression()
boston_lm.fit(train_X, train_y)

# Display intercept and regression coefficients. Round
# them to 2 decimals.
print('Regression Model for Boston Housing Training Set')
print()
print('Intercept ', np.round(boston_lm.intercept_, decimals=2))
print(pd.DataFrame({'Predictor': X.columns, 'Coefficient': np.round(boston_lm.coef_, decimals=2)}))


Regression Model for Boston Housing Training Set

Intercept  48.62
       Predictor  Coefficient
0          CRIME        -0.15
1           ZONE        -0.01
2         INDUST         0.13
3      NIT_OXIDE       -17.86
4          ROOMS         0.33
5            AGE        -0.01
6       DISTANCE        -0.66
7         RADIAL         0.22
8            TAX        -0.01
9       ST_RATIO        -0.63
10      LOW_STAT        -0.47
11    CHAR_RIV_Y         2.33
12  C_MVALUE_Yes        12.13


## Identify and compare performance measures for training and validation set.

In [9]:
# Use predict() function to make predictions for
# training set.
pred_y = boston_lm.predict(train_X)

# Create prediction performance measures for training set.
r2 = round(r2_score(train_y, pred_y),3)
adj_r2 = round(adjusted_r2_score(train_y, pred_y, boston_lm),3)

# Display prediction performance measures for training set.
print('Prediction Performance Measures for Training Set')
print('r2 : ', r2)
print('Adjusted r2 : ', adj_r2)


# Use predict() to score (make) predictions for validation set.
boston_lm_pred = boston_lm.predict(valid_X)

# Create prediction performance measures for validation set.
r2 = round(r2_score(valid_y, boston_lm_pred),3)
adj_r2 = round(adjusted_r2_score(valid_y, boston_lm_pred, boston_lm),3)

# Display prediction performance measures for validation set.
print() 
print('Prediction Performance Measures for Validation Set')
print('r2 : ', r2)
print('adjusted r2 : ', adj_r2)

Prediction Performance Measures for Training Set
r2 :  0.83
Adjusted r2 :  0.824

Prediction Performance Measures for Validation Set
r2 :  0.852
adjusted r2 :  0.838


In [10]:
# Display common accuracy measures for training set.
print('Accuracy Measures for Training Set - All Variables')
regressionSummary(train_y, pred_y)
print()

# Display common accuracy measures for validation set.
print('Accuracy Measures for Validation Set - All Variables')
regressionSummary(valid_y, boston_lm_pred)

Accuracy Measures for Training Set - All Variables

Regression statistics

                      Mean Error (ME) : 0.0000
       Root Mean Squared Error (RMSE) : 3.7145
            Mean Absolute Error (MAE) : 2.6931
          Mean Percentage Error (MPE) : -2.7567
Mean Absolute Percentage Error (MAPE) : 13.2197

Accuracy Measures for Validation Set - All Variables

Regression statistics

                      Mean Error (ME) : 0.3667
       Root Mean Squared Error (RMSE) : 3.6868
            Mean Absolute Error (MAE) : 2.7428
          Mean Percentage Error (MPE) : -2.9628
Mean Absolute Percentage Error (MAPE) : 13.9356


## Exhaustive Search algorithm.

In [14]:
# Define train_model() function used in Executive Search
# algorithm with executive_search() function. 
def train_model(variables):
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

# Define score_model() function used in Executive Search
# algorithm with executive_search() function. 
def score_model(model, variables):
    pred_y = model.predict(train_X[variables])
    # Apply negative sign as score is optimized to be 
    # as low as possible in exhaustive_search() function.
    return -adjusted_r2_score(train_y, pred_y, model)

# Create allVariables object with predcitors in train_X,
# i.e., training data set, with 13 predictor columns and 
# 304 records.
allVariables = train_X.columns

# The exhaustive_search() function consists of 3 arguments:
# - allVariables - list of all variables in training data set,
# - train_model() function that creates a model for a specific 
#    combination of variables,
# - score_model() function that score the model performance using
#     adjusted_r2.
results = exhaustive_search(allVariables, train_model, score_model)

# Create data[] loop process to identify the best model for each
# combination of 1, 2, 3, ..., 13 variables. 
data = []
for result in results:
    model = result['model']
    variables = result['variables']
    AIC = AIC_score(train_y, model.predict(train_X[variables]), model)
    
    d = {'n': result['n'], 'r2adj': -result['score'], 'AIC': AIC}
    d.update({var: var in result['variables'] for var in allVariables})
    data.append(d)

# Define the width of output presentation to be wider to
# to display results in two rows (instead of more rows
# otherwise). 
pd.set_option('display.width', 100)

# Display the exhaustive search results.
print(pd.DataFrame(data, columns=('n', 'r2adj', 'AIC') + tuple(sorted(allVariables))))

# Reset the output width to the default. 
pd.reset_option('display.width')

     n     r2adj          AIC    AGE  CHAR_RIV_Y  CRIME  C_MVALUE_Yes  DISTANCE  INDUST  LOW_STAT  \
0    1  0.615757  2227.470343  False       False  False          True     False   False     False   
1    2  0.784502  2023.736517  False       False  False          True     False   False      True   
2    3  0.793737  2009.222342  False       False   True          True     False   False      True   
3    4  0.800829  1997.822810  False        True   True          True     False   False      True   
4    5  0.804618  1992.008003  False       False  False          True      True   False      True   
5    6  0.811403  1980.477479  False        True  False          True      True   False      True   
6    7  0.816868  1971.047129  False        True   True          True      True   False      True   
7    8  0.822139  1961.682655  False        True   True          True      True   False      True   
8    9  0.822845  1961.248007  False        True   True          True      True    True    

In [17]:
# Develop the multiple linear regression model based
# on the Exhaustive Search results.

# Identify predictors and outcome of the regression model.
predictors_ex = ['CHAR_RIV_Y', 'CRIME', 'C_MVALUE_Yes',
                 'DISTANCE', 'INDUST', 'LOW_STAT', 'NIT_OXIDE', 
                 'RADIAL', 'ST_RATIO',  'TAX']
outcome = 'MVALUE'

# Identify X and y variables for regression and partition data
# using 70% of records for training and 30% for validation 
# (test_size=0.3). 
X = boston_df[predictors_ex]
y = boston_df[outcome]
train_X_ex, valid_X_ex, train_y_ex, valid_y_ex = \
          train_test_split(X, y, test_size=0.3, random_state=1)

# Create multiple linear regression model using X and y.
boston_ex = LinearRegression()
boston_ex.fit(train_X_ex, train_y_ex)

# Display intercept and regression coefficients. Round them
# to 2 decimals.
print('Regression Model for Training Set Using Exhaustive Search')
print()
print('Intercept ', round(boston_ex.intercept_, 2))
print(pd.DataFrame({'Predictor': X.columns,
            'Coefficient': np.round(boston_ex.coef_, decimals=2)}))


Regression Model for Training Set Using Exhaustive Search

Intercept  50.82
      Predictor  Coefficient
0    CHAR_RIV_Y         2.34
1         CRIME        -0.15
2  C_MVALUE_Yes        12.19
3      DISTANCE        -0.69
4        INDUST         0.13
5      LOW_STAT        -0.49
6     NIT_OXIDE       -18.39
7        RADIAL         0.23
8      ST_RATIO        -0.63
9           TAX        -0.01


In [18]:
# Use predict() function to score (make) predictions 
# for validation set and measure their accuracy using
# Exhaustive Search algorithm.
boston_ex_pred = boston_ex.predict(valid_X_ex)

# Display common accuracy measures for validation set.
print()
print('Accuracy Measures for Validation Set Using Exhaustive Search')
regressionSummary(valid_y_ex, boston_ex_pred)


Accuracy Measures for Validation Set Using Exhaustive Search

Regression statistics

                      Mean Error (ME) : 0.3854
       Root Mean Squared Error (RMSE) : 3.7318
            Mean Absolute Error (MAE) : 2.7591
          Mean Percentage Error (MPE) : -2.8698
Mean Absolute Percentage Error (MAPE) : 13.9371


##  Backward Elimination algorithm.

In [31]:
# Define train_model() function used in Backward Elimination
# algorithm with backward_elimination() function. 
def train_model(variables):
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

# Define score_model() function used in Backward Elimination
# algorithm with backward_elimination() function. 
def score_model(model, variables):
    return AIC_score(train_y, model.predict(train_X[variables]), model)

# Use backward_elimination() function to identify the
# best_model and best_variables. 
best_model_be, best_variables_be = backward_elimination(train_X.columns, 
                        train_model, score_model, verbose=True)

# Display best variables based on Backward Elimination algorithm. 
print()
print('Best Variables from Backward Elimination Algorithm')
print(best_variables_be)

Variables: CRIME, ZONE, INDUST, NIT_OXIDE, ROOMS, AGE, DISTANCE, RADIAL, TAX, ST_RATIO, LOW_STAT, CHAR_RIV_Y, C_MVALUE_Yes
Start: score=1963.68
Step: score=1962.03, remove AGE
Step: score=1960.30, remove ZONE
Step: score=1958.80, remove ROOMS
Step: score=1958.80, remove None

Best Variables from Backward Elimination Algorithm
['CRIME', 'INDUST', 'NIT_OXIDE', 'DISTANCE', 'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'CHAR_RIV_Y', 'C_MVALUE_Yes']


In [32]:
# Develop the multiple linear regression model based
# on the Backward Elimination results.

# Identify predictors and outcome of the regression model.
predictors_be = ['CRIME', 'INDUST', 'NIT_OXIDE', 'DISTANCE', 'RADIAL',
          'TAX', 'ST_RATIO', 'LOW_STAT', 'CHAR_RIV_Y', 'C_MVALUE_Yes']
               
outcome = 'MVALUE'

# Identify X and y variables for regression and partition data
# using 70% of records for training and 30% for validation 
# (test_size=0.3). 
X = boston_df[predictors_be]
y = boston_df[outcome]
train_X_be, valid_X_be, train_y_be, valid_y_be = \
          train_test_split(X, y, test_size=0.3, random_state=1)

# Create multiple linear regression model using X and y.
boston_be = LinearRegression()
boston_be.fit(train_X_be, train_y_be)

# Display intercept and regression coefficients. Round them
# to 2 decimals.
print('Regression Model for Training Set Using Backward Elimination')
print()
print('Intercept ', round(boston_be.intercept_, 2))
print(pd.DataFrame({'Predictor': X.columns,
            'Coefficient': np.round(boston_be.coef_, decimals=2)}))


Regression Model for Training Set Using Backward Elimination

Intercept  50.82
      Predictor  Coefficient
0         CRIME        -0.15
1        INDUST         0.13
2     NIT_OXIDE       -18.39
3      DISTANCE        -0.69
4        RADIAL         0.23
5           TAX        -0.01
6      ST_RATIO        -0.63
7      LOW_STAT        -0.49
8    CHAR_RIV_Y         2.34
9  C_MVALUE_Yes        12.19


In [33]:
# Use predict() to score predictions for validation set.
best_be_pred = boston_be.predict(valid_X_be)

# Display common accuracy measures for validation set.
print()
print('Accuracy Measures for Validation Set Using Forward Selection')
regressionSummary(valid_y_be, best_be_pred)


Accuracy Measures for Validation Set Using Forward Selection

Regression statistics

                      Mean Error (ME) : 0.3854
       Root Mean Squared Error (RMSE) : 3.7318
            Mean Absolute Error (MAE) : 2.7591
          Mean Percentage Error (MPE) : -2.8698
Mean Absolute Percentage Error (MAPE) : 13.9371
